In [1]:
%use adventOfCode

In [2]:
val puzzle = AocClient.fromEnv().interactiveDay(2024, 23)
puzzle.viewPartOne()

--- Day 23: LAN Party --- 
 As The Historians wander around a secure area at Easter Bunny HQ, you come across posters for a LAN party scheduled for today! Maybe you can find it; you connect to a nearby datalink port and download a map of the local network (your puzzle input). 
 The network map provides a list of every connection between two computers . For example: 
 kh-tc
qp-kh
de-cg
ka-co
yn-aq
qp-ub
cg-tb
vc-aq
tb-ka
wh-tc
yn-cg
kh-ub
ta-co
de-co
tc-td
tb-wq
wh-td
ta-ka
td-qp
aq-cg
wq-ub
ub-vc
de-ta
wq-aq
wq-vc
wh-yn
ka-de
kh-ta
co-tc
wh-qp
tb-vc
td-yn
 
 Each line of text in the network map represents a single connection; the line kh-tc represents a connection between the computer named kh and the computer named tc . Connections aren't directional; tc-kh would mean exactly the same thing. 
 LAN parties typically involve multiplayer games, so maybe you can locate it by finding groups of connected computers. Start by looking for sets of three computers where each computer in the set is connected to the other two computers. 
 In this example, there are 12 such sets of three inter-connected computers: 
 aq,cg,yn
aq,vc,wq
co,de,ka
co,de,ta
co,ka,ta
de,ka,ta
kh,qp,ub
qp,td,wh
tb,vc,wq
tc,td,wh
td,wh,yn
ub,vc,wq
 
 If the Chief Historian is here, and he's at the LAN party, it would be best to know that right away. You're pretty sure his computer's name starts with t , so consider only sets of three computers where at least one computer's name starts with t . That narrows the list down to 7 sets of three inter-connected computers: 
 co,de, ta 
co,ka, ta 
de,ka, ta 
qp, td ,wh
 tb ,vc,wq
 tc , td ,wh
 td ,wh,yn
 
 Find all the sets of three inter-connected computers. How many contain at least one computer with a name that starts with t ?

In [14]:
val testInput = """kh-tc
qp-kh
de-cg
ka-co
yn-aq
qp-ub
cg-tb
vc-aq
tb-ka
wh-tc
yn-cg
kh-ub
ta-co
de-co
tc-td
tb-wq
wh-td
ta-ka
td-qp
aq-cg
wq-ub
ub-vc
de-ta
wq-aq
wq-vc
wh-yn
ka-de
kh-ta
co-tc
wh-qp
tb-vc
td-yn"""

fun parse(input: String) = input.lines().map { it.split('-') }.flatMap { (a, b) -> listOf(a to b, b to a) }
    .groupBy({ it.first }, { it.second })
parse(testInput)

{kh=[tc, qp, ub, ta], tc=[kh, wh, td, co], qp=[kh, ub, td, wh], de=[cg, co, ta, ka], cg=[de, tb, yn, aq], ka=[co, tb, ta, de], co=[ka, ta, de, tc], yn=[aq, cg, wh, td], aq=[yn, vc, cg, wq], ub=[qp, kh, wq, vc], tb=[cg, ka, wq, vc], vc=[aq, ub, wq, tb], wh=[tc, td, yn, qp], ta=[co, ka, de, kh], td=[tc, wh, qp, yn], wq=[tb, ub, aq, vc]}

In [15]:
fun findTriplets(parsed: Map<String, List<String>>): List<List<String>> {
    fun connections(node: String) = parsed.getOrDefault(node, emptyList())
    return parsed.flatMap { (node1, connections) ->
        connections.filter { node2 -> node1 < node2 }.flatMap { node2 ->
            connections(node2).filter { node3 -> node2 < node3 }.filter { node3 -> node1 in connections(node3) }
                .map { node3 -> listOf(node1, node2, node3) }
        }
    }
}

fun part1(input: String): Int {
    val parsed = parse(input)
    val triplets = findTriplets(parsed)
    return triplets.count { triplet -> triplet.any { it[0] == 't' } }
}

part1(testInput)

7

In [16]:
part1(puzzle.input())

893

In [17]:
findTriplets(parse(puzzle.input())).sortedBy { it[0] }

[[aa, ri, wm], [aa, ri, wa], [aa, ri, xc], [aa, mb, ri], [aa, mb, wm], [aa, mb, wa], [aa, mb, qk], [aa, mb, om], [aa, mb, xc], [aa, fy, mb], [aa, fy, wa], [aa, fy, xc], [aa, fy, om], [aa, fy, le], [aa, fy, ri], [aa, fy, qk], [aa, wm, xc], [aa, eq, mb], [aa, eq, xc], [aa, eq, wa], [aa, eq, ri], [aa, eq, fy], [aa, eq, om], [aa, eq, fh], [aa, eq, wm], [aa, eq, qk], [aa, eq, le], [aa, qk, xc], [aa, qk, wa], [aa, qk, ri], [aa, qk, wm], [aa, om, xc], [aa, om, ri], [aa, om, wm], [aa, om, wa], [aa, om, qk], [aa, dl, qk], [aa, dl, om], [aa, dl, mb], [aa, dl, fh], [aa, dl, xc], [aa, dl, le], [aa, dl, ri], [aa, dl, eq], [aa, dl, fy], [aa, dl, wa], [aa, dl, wm], [aa, fh, wm], [aa, fh, om], [aa, fh, ri], [aa, fh, fy], [aa, fh, qk], [aa, fh, le], [aa, fh, xc], [aa, fh, wa], [aa, fh, mb], [aa, le, xc], [aa, le, qk], [aa, le, wm], [aa, le, mb], [aa, le, ri], [aa, le, om], [aa, le, wa], [aa, wa, wm], [aa, wa, xc], [ab, qm, xn], [ab, qm, uq], [ab, qm, ww], [ab, ww, xn], [ab, hu, or], [ab, hu, qm], [ab, 

In [18]:
class GroupFinder(val graph: Map<String, List<String>>) {
    fun connections(node: String) = graph.getOrDefault(node, emptyList())

    fun findGroups(size: Int): List<List<String>> {
        return graph.flatMap { (node1, connections) ->
            expandGroup(node1, listOf(node1), size - 1)
        }
    }

    fun expandGroup(lastNode: String, allNodes: List<String>, remainingSize: Int): List<List<String>> {
        if (remainingSize == 0) return listOf(allNodes)
        return connections(lastNode).filter { newNode -> lastNode < newNode }
            .filter { newNode -> connections(newNode).containsAll(allNodes) }.flatMap { newNode ->
            expandGroup(newNode, allNodes + newNode, remainingSize - 1)
        }
    }
}

fun part2(input: String): List<String> {
    val parsed = parse(input)
    val finder = GroupFinder(parsed)
    return (0..parsed.size).asSequence().map {
        finder.findGroups(it)
    }.first { it.size == 1 }.single()
}



In [19]:
part2(testInput)

[co, de, ka, ta]

In [21]:
part2(puzzle.input()).joinToString(",")

cw,dy,ef,iw,ji,jv,ka,ob,qv,ry,ua,wt,xz